In [161]:
#Deploy (see: https://cloud.google.com/ai-platform/prediction/docs/deploying-models?authuser=2)
MODEL_VERSION="version4"
MODEL_NAME="kubeflow_test"
MODEL_PATH="gs://ai-vqc-kubeflow-output/model" #it must contain a Tensorflow model saved in SavedModel directory
FRAMEWORK="tensorflow" #omit for custom deploy
MODEL_PYTHON_VERSION="3.5"
MODEL_REGION="europe-west1"
RUNTIME_VERSION="1.14"

In [ ]:
! gcloud ai-platform models create $MODEL_NAME --regions $REGION

In [ ]:
#for tensorflow deploy
! gcloud ai-platform versions create $MODEL_VERSION \
--model=$MODEL_NAME \
--origin=$MODEL_PATH \
--framework=$FRAMEWORK \
--python-version=$MODEL_PYTHON_VERSION \
--runtime-version=$RUNTIME_VERSION

In [199]:
#for custom routines
PROJECT_ID='ai-vqc'
MODEL_NAME="kubeflow_test_log"
MODEL_VERSION="version1"
MODEL_PATH="gs://ai-vqc-kubeflow-output/model/"
MODEL_PYTHON_VERSION="3.5"
MODEL_REGION="europe-west1"
RUNTIME_VERSION="1.14"
MODEL_PACKAGE_URIS="gs://ai-vqc-kubeflow-components/routine/custom_prediction_routine-0.2.tar.gz" #use for custom deploy
MODEL_PREDICTION_CLASS="model_prediction.CustomModelPrediction" #the name of your Predictor class

In [200]:
#for custom routines
! gcloud beta ai-platform models create $MODEL_NAME --regions $MODEL_REGION --enable-console-logging

Created ml engine model [projects/ai-vqc/models/kubeflow_test_log].


In [201]:
#for custom routines
#! gcloud components install beta
! gcloud beta ai-platform versions create $MODEL_VERSION \
--model $MODEL_NAME \
--origin $MODEL_PATH \
--runtime-version=$RUNTIME_VERSION \
--python-version=$MODEL_PYTHON_VERSION \
--package-uris=$MODEL_PACKAGE_URIS \
--prediction-class=$MODEL_PREDICTION_CLASS

Creating version (this might take a few minutes)......done.                    


In [204]:
#Invoke predictions (Bash)

data='{"instances": [[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12]]}'
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
INPUT_DATA_FILE='data.json'
! gcloud beta ai-platform predict --model $MODEL_NAME --version $MODEL_VERSION --json-instances $INPUT_DATA_FILE

{
  "error": "Prediction failed: unknown error."
}


In [196]:
#Invoke prediction (Python)

#!pip install --upgrade google-api-python-client

import googleapiclient.discovery

instances = [
  [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12],
  [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12],
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, MODEL_VERSION)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()


if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

RuntimeError: Prediction failed: unknown error.

In [184]:
#TEST LOCAL
%mkdir /home/jovyan/local_test
%cd /home/jovyan/local_test

#read data
PROJECT='ai-vqc'
BUCKET_NAME='ai-vqc-kubeflow-output/model'
PREFIX=''
FILENAME='keras_saved_model.h5'

!gsutil cp gs://$BUCKET_NAME/$FILENAME .
!ls -al

import os
import pickle
import numpy as np
import tensorflow as tf

class CustomModelPrediction():

  def __init__(self, model):
    self._model = model
    #self._processor = processor
  
  '''
  def postprocess(self, predictions):
    labeled_predictions = []

    for prediction in predictions:
      labeled_prediction = []
      for word_prediction in prediction:
        labeled_prediction.append(self._processor.labels[word_prediction])
      labeled_predictions.append(labeled_prediction)

    return labeled_predictions
  '''
  def predict(self, instances):
    #transformed_instances = self._processor.transform(instances)
    #predictions = self._model.predict(np.array(transformed_instances))
    predictions = self._model.predict(np.array(instances))
    #predictions = np.argmax(predictions, axis=-1).tolist()

    #labels = self.postprocess(predictions)
    #return labels
    return predictions

  @classmethod
  def from_path(cls, model_dir):
    model = tf.keras.models.load_model(os.path.join(model_dir, 'keras_saved_model.h5'))
    #with open(os.path.join(model_dir, 'preprocessor.pkl'), 'rb') as f:
    #  processor = pickle.load(f)

    return cls(model)

predictor_class = CustomModelPrediction(model)

instances = [ #In keras, a None dimension means that it can be any scalar number, it just denotes that you are free to select the length (number of samples) of your input during testing
  [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12],
  [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12],
]
predictor_class.predict(instances)

In [154]:
#Export preprocess files
import pickle
scaler = MySimpleScaler()
with open ('preprocessor.pkl', 'wb') as f:
  pickle.dump(scaler, f)
!gsutil cp preprocessor.pkl gs://$BUCKET_NAME

Copying file://preprocessor.pkl [Content-Type=application/octet-stream]...
/ [1 files][   67.0 B/   67.0 B]                                                
Operation completed over 1 objects/67.0 B.                                       
